In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from typing import List

In [2]:
df = pd.read_csv('data_set.csv', index_col='Date')

In [3]:
# code used with https://colab.research.google.com/drive/1MW1s8N5Q2jNOVxo5juYww-r1ET7-wi32?usp=sharing
# https://www.youtube.com/watch?v=LI94ZkjE_w4

In [4]:
df.shape

(6973, 35)

In [5]:
df.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA12,MA26,MA50,...,% day (ma12&ma50) (cd 12&50),% day (ma12&ma100) (cd 12&100),% day (ma12&ma200) (cd 12&200),EIS,EE centerline,EE myvar,EE myvars,EE mymov,EE Above,EE Belowe
Date,,,,,,,,,,,,,,,,,,,,,
1993-11-11,27.799976,27.874707,27.706562,27.725245,88900,0.0,0,27.781295,27.797104,27.643933,...,0.496899,2.002303,3.719210,1,27.797104,0.077602,0.006022,0.307292,28.411687,27.182521
1993-11-12,27.781308,27.949453,27.762625,27.856039,108200,0.0,0,27.784410,27.803572,27.646631,...,0.498357,1.966286,3.692839,2,27.803572,0.145881,0.021281,0.307538,28.418648,27.188496
1993-11-15,27.912078,27.912078,27.762616,27.837347,243300,0.0,0,27.790637,27.816506,27.654544,...,0.492120,1.930584,3.675571,2,27.816506,0.095572,0.009134,0.307678,28.431863,27.201150
1993-11-16,27.893405,27.986819,27.781308,27.968136,492600,0.0,0,27.782854,27.828003,27.661340,...,0.439289,1.853965,3.609263,2,27.828003,0.158815,0.025222,0.308037,28.444077,27.211930
1993-11-17,27.986798,27.986798,27.743922,27.818653,39600,0.0,0,27.784411,27.841656,27.673718,...,0.399991,1.813010,3.578146,2,27.841656,0.145142,0.021066,0.307174,28.456003,27.227308


In [6]:
# df[['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
#        'MA12', 'MA26', 'MA50', 'MA100', 'MA200', 'MA13', 'std12', 'std26',
#        'std50', 'std100', 'std200', '% day (o&c)', '% day (l&h)',
#        '% change (o&c) & (l&h)', '% day (ma12&ma26) (cd 12&26)',
#        '% day (ma26&ma50) (cd 26&50)', '% day (ma50&ma100) (cd 50&100)',
#        '% day (ma100&ma200) (cd 100&200)', '% day (ma12&ma50) (cd 12&50)',
#        '% day (ma12&ma100) (cd 12&100)', '% day (ma12&ma200) (cd 12&200)',
#        'EIS', 'EE centerline', 'EE myvar', 'EE myvars', 'EE mymov', 'EE Above',
#        'EE Belowe']].corr()

In [7]:
df = df.reset_index()

In [8]:
# Columns Date assign type datetime64[ns]
from datetime import datetime
from functools import partial
to_datetime_fmt = partial(pd.to_datetime, format='%Y-%M-%d')
df['Date'] = df['Date'].apply(to_datetime_fmt)

In [9]:
# print(df.dtypes)
# df

In [10]:
df_6_yr = df[df["Date"] > df["Date"].max() - timedelta(days=365 * 6)]

In [11]:
df_6_yr.shape

(1398, 36)

In [12]:
df_6_yr["Date"].min(), df_6_yr["Date"].max()

(Timestamp('2016-01-01 00:02:00'), Timestamp('2021-01-31 00:03:00'))

In [13]:
train_size = int(df_6_yr.shape[0] * 0.8)
train_df = df_6_yr.iloc[:train_size]
val_df = df_6_yr.iloc[train_size:]

In [14]:
train_df.shape, val_df.shape

((1118, 36), (280, 36))

In [15]:
train_df["Date"].min(), train_df["Date"].max(), val_df["Date"].min(), val_df["Date"].max()

(Timestamp('2016-01-01 00:02:00'),
 Timestamp('2020-01-31 00:03:00'),
 Timestamp('2020-01-01 00:07:00'),
 Timestamp('2021-01-31 00:03:00'))

In [16]:
scaler = StandardScaler()
scaler.fit(train_df[["Low"]])

def make_dataset(
    df,
     window_size, 
     batch_size,
     use_scaler=True,
     shuffle=True
     ):
  features = df[["Low"]].iloc[:-window_size]
  if use_scaler:
    features = scaler.transform(features)
  data = np.array(features, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=df["Low"].iloc[window_size:],
      sequence_length=window_size,
      sequence_stride=1,
      shuffle=shuffle,
      batch_size=batch_size)
  return ds

In [17]:
example_ds = make_dataset(df=train_df, window_size=3, batch_size=2, use_scaler=False, shuffle=False)

AttributeError: module 'tensorflow_core.keras.preprocessing' has no attribute 'timeseries_dataset_from_array'

In [ ]:
example_feature, example_label = next(example_ds.as_numpy_iterator())

In [ ]:
example_feature.shape


In [ ]:
example_label.shape


In [ ]:
train_df["Low"].iloc[:6]

In [ ]:
print(example_feature[0])
print(example_label[0])

In [ ]:
print(example_feature[1])
print(example_label[1])

In [ ]:
window_size = 10
batch_size = 8
train_ds = make_dataset(df=train_df, window_size=window_size, batch_size=batch_size, use_scaler=True, shuffle=True)
val_ds = make_dataset(df=val_df, window_size=window_size, batch_size=batch_size, use_scaler=True, shuffle=True)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(1)
])

In [ ]:
def compile_and_fit(model, train_ds, val_ds, num_epochs: int = 20):
  model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError()]
      )
  history = model.fit(
      train_ds, 
      epochs=num_epochs,
      validation_data=val_ds,
      verbose=0
      )
  return history

In [ ]:
history =  compile_and_fit(lstm_model, train_ds, val_ds, num_epochs=100)

In [ ]:
plt.plot(history.history['mean_absolute_error'])

In [ ]:
plt.plot(history.history['val_mean_absolute_error'])

In [ ]:
lstm_model.evaluate(train_ds)

In [ ]:
lstm_model.evaluate(val_ds)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(1)
])

history =  compile_and_fit(lstm_model, train_ds, val_ds, num_epochs=500)

In [ ]:
lstm_model.evaluate(train_ds)

In [ ]:
lstm_model.evaluate(val_ds)

In [ ]:
plt.plot(history.history['mean_absolute_error'])

In [ ]:
plt.plot(history.history['val_mean_absolute_error'])

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

history =  compile_and_fit(lstm_model, train_ds, val_ds, num_epochs=500)

In [ ]:
lstm_model.evaluate(train_ds)

In [ ]:
lstm_model.evaluate(val_ds)

In [ ]:
plt.plot(history.history['mean_absolute_error'])

In [ ]:
plt.plot(history.history['val_mean_absolute_error'])